In [1]:
import sys, os, pickle
from joblib import Parallel, delayed
import pandas as pd
from jumpmodels.utils import filter_date_range
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd
from jumpmodels.sparse_jump import SparseJumpModel
from sklearn.preprocessing import StandardScaler
from pypfopt.black_litterman import BlackLittermanModel, market_implied_prior_returns
from pypfopt.efficient_frontier import EfficientFrontier
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import exceptions 
from scipy.optimize import brentq
import cvxpy as cp
from scipy import stats

#sys.path.append('/Users/victor/Documents/thesis_vri_vp/vic_new')         # for mac
sys.path.append('C:\\Users\\victo\\git_new\\thesis_vri_vp\\vic_new')      # for windows
# sys.path.append('/Users/vlad/Desktop/git/Masters-Thesis-VRI-VP/vic_new')         # for mac vlad
from feature_set_v2 import MergedDataLoader 

In [ ]:

REFIT_FREQ        = "ME"        
MIN_TRAINING_YEARS= 8
MAX_TRAINING_YEARS= 12
INITIAL_TRAIN_START = "2002-05-31"
test_start        = "2017-01-01"


cv_choice = "bayes"

script_dir = os.getcwd()
base_dir   = os.path.abspath(os.path.join(script_dir, "..", "..", "..","..", "..",".."))
data_dir   = os.path.join(base_dir, "data_new")

factor_file = os.path.join(data_dir, "1estimation_index_returns.csv")
market_file = os.path.join(data_dir, "1macro_data.csv")
etf_file    = os.path.join(data_dir, "2trading_etf_returns_aligned.csv")

factors = ["iwf", "mtum", "qual", "size", "usmv", "vlue"]  


bayes_df   = pd.read_parquet("cv_params_bayes_v2.parquet") 


In [ ]:

df_map = {

    "bayes":   bayes_df,

}
cv_df = df_map[cv_choice]


SMOOTH_METHOD = "none"  
SMOOTH_WINDOW = 3       


cv_df = cv_df.sort_values(["factor","date"])

if SMOOTH_METHOD == "none":

    cv_df["sm_lambda"] = cv_df["best_lambda"]
    cv_df["sm_kappa"]  = cv_df["best_kappa"]

elif SMOOTH_METHOD == "rolling_median":

    cv_df["sm_lambda"] = (
        cv_df
        .groupby("factor")["best_lambda"]
        .transform(lambda x: x.rolling(SMOOTH_WINDOW, min_periods=1, center=True).median())
    )
    cv_df["sm_kappa"] = (
        cv_df
        .groupby("factor")["best_kappa"]
        .transform(lambda x: x.rolling(SMOOTH_WINDOW, min_periods=1, center=True).median())
    )

elif SMOOTH_METHOD == "ewma":

    cv_df["sm_lambda"] = (
        cv_df
        .groupby("factor")["best_lambda"]
        .transform(lambda x: x.ewm(span=SMOOTH_WINDOW, min_periods=1).mean())
    )
    cv_df["sm_kappa"] = (
        cv_df
        .groupby("factor")["best_kappa"]
        .transform(lambda x: x.ewm(span=SMOOTH_WINDOW, min_periods=1).mean())
    )

else:
    raise ValueError(f"Unknown SMOOTH_METHOD {SMOOTH_METHOD!r}")


cv_df["sm_kappa"] = cv_df["sm_kappa"].round().astype(int)


cv_df["best_lambda"] = cv_df["sm_lambda"]
cv_df["best_kappa"]  = cv_df["sm_kappa"]


cv_df.drop(columns=["sm_lambda","sm_kappa"], inplace=True)


saved_hyperparams = {}
for fac in factors:
    sub = cv_df[cv_df["factor"] == fac].sort_values("date")
    saved_hyperparams[fac] = [
        {
            "date":      row["date"],
            "new_lambda": row["best_lambda"],
            "new_kappa":  row["best_kappa"]
        }
        for _, row in sub.iterrows()
    ]


In [ ]:

factor_data_dict  = {}
factor_returns_ls = []

for fac in factors:
    print(f"Loading data for {fac}")
    data = MergedDataLoader(
        factor_file=factor_file,
        market_file=market_file,
        ver="v2",
        factor_col=fac
    ).load()

    common_idx = (data.X.index
                  .intersection(data.ret_ser.index)
                  .intersection(data.market_ser.index))

    X_full        = data.X.loc[common_idx]
    fac_ret_full  = data.ret_ser.loc[common_idx]
    mkt_ret_full  = data.market_ser.loc[common_idx]
    active_ret    = fac_ret_full - mkt_ret_full

    factor_data_dict[fac] = {
        "X"        : X_full,
        "fac_ret"  : fac_ret_full,
        "mkt_ret"  : mkt_ret_full,
        "active_ret": active_ret,
    }
    factor_returns_ls.append(fac_ret_full)

all_market_ret = mkt_ret_full


full_factors_df = pd.concat(factor_returns_ls, axis=1).dropna()
full_df = pd.concat([full_factors_df, all_market_ret], axis=1).dropna()
full_df.columns = factors + ["Market"]


etf_df   = pd.read_csv(etf_file, index_col=0, parse_dates=True).dropna().sort_index()
rf_ser   = etf_df["rf"]
full_df  = pd.concat([full_df, rf_ser], axis=1).dropna()
full_df.columns = factors + ["Market", "rf"]


test_slice = full_df.loc[test_start:]
test_index = test_slice.index.sort_values()



Loading data for iwf
Loading data for mtum
Loading data for qual
Loading data for size
Loading data for usmv
Loading data for vlue


In [ ]:

VIEWS_FILE = "views2.pkl"
FORCE_REBUILD = False 

def _fit_one_factor(fac, refit_date, test_dates_chunk,
                    factor_data_dict, hyperparams,
                    min_years, max_years, init_start):


    def get_train_window(current_date, full_data):
        train_end  = current_date
        train_start= max(train_end - pd.DateOffset(years=max_years),
                         pd.to_datetime(init_start))
        if (train_end - train_start) < pd.Timedelta(days=365.25*min_years):
            train_start = train_end - pd.DateOffset(years=min_years)
        idx = full_data.index
        subset = idx[(idx >= train_start) & (idx <= train_end)]
        start_date, end_date = subset.min(), subset.max()
        return start_date, end_date 


    fac_data = factor_data_dict[fac]
    X   = fac_data["X"]
    ret = fac_data["fac_ret"]
    act = fac_data["active_ret"]

    lam = hyperparams["new_lambda"]
    kp  = hyperparams["new_kappa"]
    train_start, train_end = get_train_window(refit_date, X)


    clipper = DataClipperStd(mul=3.0)
    scaler  = StandardScaler()
    X_train = scaler.fit_transform(clipper.fit_transform(
                 filter_date_range(X, train_start, train_end)))
    active_train = filter_date_range(act, train_start, train_end)


    sjm = SparseJumpModel(n_components=2,
                          max_feats=int(kp**2),
                          jump_penalty=lam)
    
    train_idx = filter_date_range(X, train_start, train_end).index
    X_train_df = pd.DataFrame(X_train, index=train_idx, columns=X.columns)
    sjm.fit(X_train_df, ret_ser=active_train, sort_by="cumret")

    ret_train = filter_date_range(ret, train_start, train_end)


    train_states = sjm.predict(X_train_df)
    abs_ret = {}
    for st in range(2):
        st_idx = (train_states==st)
        abs_ret[st] = ret_train.loc[st_idx].mean() * 252


    states = {}
    for day in test_dates_chunk:
        X_hist = X.loc[:day]             
        temp_clipper = DataClipperStd(mul=3.0)
        X_hist_clip  = temp_clipper.fit_transform(X_hist)

        temp_scaler  = StandardScaler()
        _ = temp_scaler.fit_transform(X_hist_clip)  

        if day in X.index:
            X_day_clip   = temp_clipper.transform(X.loc[[day]])
            X_day_scaled = temp_scaler.transform(X_day_clip)
            states[day]  = sjm.predict_online(
                pd.DataFrame(X_day_scaled,
                            index=[day],
                            columns=X.columns)).iloc[0]


    out = pd.DataFrame({"state": pd.Series(states)},
                       index=list(states.keys()))
    out["ann_abs_ret"] = out["state"].map(abs_ret)
    return fac, out

def build_factor_views(factor_data_dict, saved_hyperparams, factors,
                       test_index,
                       refit_freq="ME", min_years=8, max_years=12,
                       init_start="2002-05-31"):

    views = {f:[] for f in factors}
    refit_dates = (test_index.to_series()
                   .resample(refit_freq)
                   .last()
                   .dropna())

    for j, refit_date in enumerate(refit_dates):
        if j < len(refit_dates)-1:
            next_refit = refit_dates.iloc[j+1]
        else:
            next_refit = test_index[-1]
        test_mask = (test_index>refit_date)&(test_index<=next_refit)
        test_chunk = test_index[test_mask]


        jobs = []
        for fac in factors:

            hp_hist = [h for h in saved_hyperparams[fac]
                       if pd.to_datetime(h["date"])<=refit_date]
            if not hp_hist: continue
            hp = hp_hist[-1]
            jobs.append(delayed(_fit_one_factor)(
                fac, refit_date, test_chunk,
                factor_data_dict, hp,
                min_years, max_years, init_start))
        for fac, df in Parallel(n_jobs=-1)(jobs):
            views[fac].append(df)


    for fac in factors:
        views[fac] = (pd.concat(views[fac])
                      .sort_index()
                      .loc[:,["state","ann_abs_ret"]])
    return views

   
if FORCE_REBUILD or not os.path.exists(VIEWS_FILE):
    factor_views = build_factor_views(factor_data_dict, saved_hyperparams, factors, 
                                      test_index,
                                      refit_freq=REFIT_FREQ, 
                                      min_years=8, max_years=12, init_start="2002-05-31")
    with open(VIEWS_FILE, "wb") as f:
        pickle.dump(factor_views, f)
else:
    with open(VIEWS_FILE, "rb") as f:
        factor_views = pickle.load(f)



In [ ]:


def ewm_covariance(returns, halflife=126, min_periods=60):
    ewm_cov = returns.ewm(halflife=halflife,
                          adjust=False,
                          min_periods=min_periods).cov()
    if returns.empty: return pd.DataFrame()
    return ewm_cov.loc[returns.index[-1]]

def detect_state_shifts(views, factors):

    state_df = pd.concat({f: views[f]["state"] for f in factors}, axis=1)

    return state_df.ne(state_df.shift()).any(axis=1)


def make_relative_views(view_dict, assets, benchmark="Market"):

    if benchmark not in assets:
        raise ValueError(f"Benchmark {benchmark!r} not in trade universe")

    n = len(assets)
    k = len(view_dict)
    P = np.zeros((k, n))
    Q = np.zeros(k)

    for i, (fac, v) in enumerate(view_dict.items()):
        if fac not in assets:
            continue             
        P[i, assets.index(fac)]       = 1
        P[i, assets.index(benchmark)] = -1
        Q[i] = v                      
    return P, Q


def bl_max_sharpe_te(cov_hist, pi, views, tau, delta,
                     w_bmk, te_target, bounds, rf,
                     use_bl_cov=False):

    P, Q = make_relative_views(views, list(w_bmk.index), benchmark="Market")
    bl   = BlackLittermanModel(cov_hist, pi=pi, tau=tau,
                           delta=delta, P=P, Q=Q)


    Sigma = bl.bl_cov().values if use_bl_cov else cov_hist.values
    mu    = bl.bl_returns().values
    n     = len(mu)

    w = cp.Variable(n)
    w_act = w - w_bmk.values

    prob = cp.Problem(
        cp.Maximize((mu - rf) @ w),
        [
            cp.sum(w) == 1,
            w >= np.array([lo for lo, hi in bounds]),
            w <= np.array([hi for lo, hi in bounds]),
            cp.quad_form(w_act, Sigma) <= te_target**2
        ]
    )
    prob.solve(solver="SCS")
    return pd.Series(w.value, index=w_bmk.index)


def run_bl_with_drift(views, returns_df, full_df,
                      shift_series=None,
                      tau=0.05, delta=2.5,
                      te_target=0.05,
                      trade_market=True,
                      use_bl_cov=False,
                      allow_market_short=False,
                      allow_factor_short=False,
                      use_bl_prior=False,
                      fallback_strategy="HOLD_RFR",
                      tcost=0.0007,
                      initial_capital=1_000_000):


    assets  = returns_df.columns.tolist()
    factors = list(views.keys())


    if trade_market:
        trade_assets = [a for a in assets if a != "rf"]
    else:
        trade_assets = [a for a in assets if a not in {"rf", "Market"}]

    cash_asset = "rf"


    bounds = []
    for a in trade_assets:
        if a == "Market":
            bounds.append((-.3, .3) if allow_market_short else (0, 0.3))
        else:
            bounds.append((-.3, .3) if allow_factor_short else (0, 0.3))


    w_view   = pd.DataFrame(0.0, index=returns_df.index, columns=trade_assets + [cash_asset], dtype=float)
    w_actual = pd.DataFrame(0.0, index=returns_df.index, columns=trade_assets + [cash_asset], dtype=float)


    positions = pd.Series(0.0, index=trade_assets + [cash_asset])
    capital   = initial_capital


    rets = pd.Series(0.0, index=returns_df.index)

    for i, t in enumerate(returns_df.index):
        old_capital = capital 

        if i > 0:
            day_ret = returns_df.loc[t, trade_assets + [cash_asset]]
            daily_pnl = (positions[trade_assets] * day_ret[trade_assets]).sum() \
                        + positions[cash_asset] * day_ret[cash_asset]

            capital += daily_pnl

            rets.loc[t] = daily_pnl / old_capital


        do_rebalance = False
        if i == 0:
            do_rebalance = True
        elif shift_series is not None and shift_series.loc[t]:
            do_rebalance = True

        if do_rebalance:
            hist = full_df[trade_assets].loc[:t].iloc[:-1]
            cov  = ewm_covariance(hist) * 252

            if cov.empty or cov.isna().any().any():
                w_view.loc[t, trade_assets] = 0.0
                w_view.loc[t, cash_asset]   = 1.0  
            else:

                if use_bl_prior:
                    market_caps  = {etf: 1.0 for etf in trade_assets}
                    prior_for_bl = market_implied_prior_returns(market_caps, delta, cov)
                else:
                    prior_for_bl = "equal"

                q = {fac: views[fac].loc[t, "active_ret"] for fac in factors}
                rf_annual = returns_df.loc[t, cash_asset] * 252


                q_abs = {fac: views[fac].loc[t, "ann_abs_ret"] for fac in factors}

                bl0 = BlackLittermanModel(
                    cov,
                    pi=prior_for_bl,
                    tau=tau,
                    delta=delta,
                    absolute_views=q_abs
)


                if (
                    fallback_strategy != "NO_FALLBACK"
                    and (bl0.bl_returns() <= rf_annual).all()
                ):

                    w_view.loc[t, trade_assets] = 0.0 
                    if fallback_strategy == "HOLD_RFR":
                        w_view.loc[t, cash_asset] = 1.0  
                    elif fallback_strategy == "SHORT_MARKET" and "Market" in trade_assets:
                        w_view.loc[t, "Market"]   = -1.0
                        w_view.loc[t, cash_asset] = 1.0

                else:

                    w_opt = bl_max_sharpe_te(
                        cov,
                        pi=prior_for_bl,
                        views=q,
                        tau=tau,
                        delta=delta,
                        w_bmk=pd.Series(1 / len(trade_assets), index=trade_assets),
                        te_target=te_target,
                        bounds=bounds,
                        rf=rf_annual,
                        use_bl_cov=use_bl_cov
                    )
                    w_view.loc[t, trade_assets] = w_opt
                    w_view.loc[t, cash_asset]   = 1 - w_opt.sum()



            new_positions = capital * w_view.loc[t]


            if tcost > 0 and i > 0:
                trades = (new_positions - positions).abs().sum()
                cost   = trades * tcost
                capital -= cost

                rets.loc[t] = (capital - old_capital) / old_capital

                if capital > 0:
                    new_positions = capital * w_view.loc[t]

            positions = new_positions

        else:
          
            if i > 0:
                w_view.loc[t] = w_view.iloc[i - 1]


        if capital > 0:
            w_actual.loc[t] = positions / capital
        else:
            w_actual.loc[t] = 0.0

    return w_view, w_actual, rets


In [ ]:

def annualized_sharpe(r):      
    return (r.mean() / r.std()) * np.sqrt(252)

def ann_turnover(w):
    daily_turn = w.diff().abs().sum(axis=1).mean()
    return daily_turn * 252

test_df = full_df.loc[test_index]
shift_days = detect_state_shifts(factor_views, factors).reindex(test_df.index, fill_value=False)
shift_days.iloc[0] = True

cfgs = [

     dict(label="Build RF dates",  tau=0.05, delta=2.5,shift_series=shift_days, trade_market=True,
         use_bl_cov=False, use_bl_prior=True, allow_market_short=False, allow_factor_short=False, te_target=0.04, fallback_strategy="HOLD_RFR"),

]


run_results = {}
for c in cfgs:
    label = c.pop("label")
    w_view, w_act, daily_returns = run_bl_with_drift(
        factor_views, 
        test_df,     
        full_df,     
        **c
    )
    
    run_results[label] = {
        "returns":        daily_returns,  
        "weights_view":   w_view,
        "weights_actual": w_act,
        "cfg":            c
    }
 




In [ ]:
wv = run_results['Build RF dates']['weights_view']
rf_dates_single = wv.index[wv['rf'] == 1.0].tolist()
print(rf_dates_single)


In [9]:
wv = run_results['Risk-off 2% TE, 7bps']['weights_view']
rf_dates_single = wv.index[wv['rf'] == 1.0].tolist()
print(rf_dates_single)


[Timestamp('2018-02-08 00:00:00'), Timestamp('2018-02-09 00:00:00'), Timestamp('2018-02-12 00:00:00'), Timestamp('2018-02-13 00:00:00'), Timestamp('2018-03-01 00:00:00'), Timestamp('2018-03-02 00:00:00'), Timestamp('2018-03-22 00:00:00'), Timestamp('2018-03-23 00:00:00'), Timestamp('2018-03-26 00:00:00'), Timestamp('2018-03-27 00:00:00'), Timestamp('2018-03-28 00:00:00'), Timestamp('2018-03-29 00:00:00'), Timestamp('2018-04-02 00:00:00'), Timestamp('2018-04-03 00:00:00'), Timestamp('2018-04-04 00:00:00'), Timestamp('2018-04-05 00:00:00'), Timestamp('2018-04-06 00:00:00'), Timestamp('2018-04-09 00:00:00'), Timestamp('2018-04-10 00:00:00'), Timestamp('2018-04-11 00:00:00'), Timestamp('2018-04-12 00:00:00'), Timestamp('2018-04-13 00:00:00'), Timestamp('2018-04-16 00:00:00'), Timestamp('2018-04-17 00:00:00'), Timestamp('2018-04-18 00:00:00'), Timestamp('2018-04-19 00:00:00'), Timestamp('2018-04-20 00:00:00'), Timestamp('2018-04-23 00:00:00'), Timestamp('2018-04-24 00:00:00'), Timestamp('20